In [21]:
from langgraph.graph import StateGraph,START,END
from langchain_huggingface import ChatHuggingFace,HuggingFaceEndpoint
from typing import TypedDict,Annotated,Literal,Union,List
from pydantic import BaseModel,Field
import operator
from langchain_core.messages import SystemMessage, HumanMessage
import json


In [31]:
llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    task="text-generation",
)
generation_model = ChatHuggingFace(llm=llm)

llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.1-8B-Instruct",
    task="text-generation",
)

evaluation_model = ChatHuggingFace(llm=llm)

llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    task="text-generation",
)

optimizer_model=ChatHuggingFace(llm=llm)


In [23]:
class TweetState(TypedDict):
    topic:str
    tweet:str
    evaluation:Literal["approved","needs_improvement"]
    feedback:str
    iteration:int
    max_iteration:int

    tweet_history:Annotated[List[str],operator.add]
    feedback_history:Annotated[List[str],operator.add]


In [24]:
class TweetEvaluation(BaseModel):
    evaluation: Literal["approved", "needs_improvement"] = Field(..., description="Final evaluation result.")
    feedback: Union[str, List[str]] = Field(..., description="feedback for the tweet.")

In [25]:
def generate_tweet(state:TweetState):
    print("geneeation")
    messages = [
        SystemMessage(content="You are a funny and clever Twitter/X influencer."),
        HumanMessage(content=f"""
Write a short, original, and hilarious tweet on the topic: "{state['topic']}".

Rules:
- Do NOT use question-answer format.
- Max 280 characters.
- Use observational humor, irony, sarcasm, or cultural references.
- Think in meme logic, punchlines, or relatable takes.
- Use simple, day to day english
""")
    ]

    response = generation_model.invoke(messages).content

    return {"tweet":response,'tweet_history':[response]}


In [26]:
def evaulate_tweet(state:TweetState):
    import time
    time.sleep(10)
    print("evaluation")
    
    messages = [
    SystemMessage(content="You are a ruthless, no-laugh-given Twitter critic. You evaluate tweets based on humor, originality, virality, and tweet format."),
    HumanMessage(content=f"""
Evaluate the following tweet:

Tweet: "{state['tweet']}"

Use the criteria below to evaluate the tweet:

1. Originality – Is this fresh, or have you seen it a hundred times before?  
2. Humor – Did it genuinely make you smile, laugh, or chuckle?  
3. Punchiness – Is it short, sharp, and scroll-stopping?  
4. Virality Potential – Would people retweet or share it?  
5. Format – Is it a well-formed tweet (not a setup-punchline joke, not a Q&A joke, and under 280 characters)?

reject if:
- It's written in question-answer format (e.g., "Why did..." or "What happens when...")
- It exceeds 280 characters
- It reads like a traditional setup-punchline joke
- Dont end with generic, throwaway, or deflating lines that weaken the humor (e.g., “Masterpieces of the auntie-uncle universe” or vague summaries)

### Respond ONLY in structured json format:
{{
    "evaluation": "approved" | "needs_improvement",
    "feedback" : feedback for the tweet
}}
""")
]
    
    response = evaluation_model.invoke(messages).content
    data = json.loads(response)
    if isinstance(data, list):
        data = data[0]

    result = TweetEvaluation(**data)

    return {'evaluation':result.evaluation,'feedback':result.feedback,'feedback_history':[result.feedback]}

In [27]:
import time
def optimize_tweet(state:TweetState):
    time.sleep(10)
    print("optimization")
    messages = [
        SystemMessage(content="You punch up tweets for virality and humor based on given feedback."),
        HumanMessage(content=f"""
Improve the tweet based on this feedback:
"{state['feedback']}"

Topic: "{state['topic']}"
Original Tweet:
{state['tweet']}

Re-write it as a short, viral-worthy tweet. Avoid Q&A style and stay under 280 characters.
""")
    ]


    response = optimizer_model.invoke(messages).content

    iteration = state['iteration']+1

    return {'tweet':response,'iteration':iteration,'tweet_history':[response]}
    

In [28]:
def route_evaluation(state:TweetState):
    if state['evaluation']=='approved' or state['iteration'] >=state['max_iteration']:
        return 'approved'
    else:
        return 'needs_improvement'

In [29]:
graph = StateGraph(TweetState)

graph.add_node('generate',generate_tweet)
graph.add_node('evaulate',evaulate_tweet)
graph.add_node('optimize',optimize_tweet)

graph.add_edge(START,'generate')
graph.add_edge('generate','evaulate')
graph.add_conditional_edges('evaulate',route_evaluation,{'approved':END,'needs_improvement':'optimize'})


graph.add_edge('optimize','generate')

workflow = graph.compile()

In [11]:
from IPython.display import Image
Image(workflow.get_graph().draw_mermaid_png())

ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph after 1 retries. To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

In [32]:
initial_state={
    "topic":'indian railway',
    'iteration':1,
    'max_iteration':3
}

workflow.invoke(initial_state)

geneeation


HfHubHTTPError: 402 Client Error: Payment Required for url: https://router.huggingface.co/featherless-ai/v1/chat/completions (Request ID: Root=1-68b774ea-1d36b5dd64bc24143d9f0fb7;2f3511cc-e2e5-49ae-8b77-ef4a051524fc)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.